In [9]:
import pickle
import torch
import numpy as np
import random as rd

In [106]:
with open('../data/tsp/val_cost.pkl', 'rb') as file:
    data = pickle.load(file)
cost = [torch.FloatTensor(x) for x in data]

with open('../data/tsp/val_location.pkl', 'rb') as file:
    data = pickle.load(file)
location = [torch.FloatTensor(x) for x in data]

In [114]:
cost = torch.stack(cost)

In [122]:
seq = []
for i in range(len(cost)):
    pi = np.arange(0, 100)
    rd.shuffle(pi)
    seq.append(pi)

pi_seq = [torch.tensor(i, dtype=torch.int64) for i in seq]
pi_seq = torch.stack(pi_seq, 0)


In [125]:
matrix_by_row = torch.gather(cost, 1, pi_seq.unsqueeze(-1).expand_as(cost))
reorder_cost_data = torch.gather(matrix_by_row, 2, pi_seq.unsqueeze(1).expand_as(cost))



In [126]:
reorder_cost_data[17][5:10][:,5:10]

tensor([[  0.0000,  43.1000,  59.7000,   0.0000,  31.3000],
        [ 81.3000,   0.0000,  16.6000,   0.0000,  69.6000],
        [ 64.7000,  64.8000,   0.0000,   0.0000,  53.0000],
        [674.7000, 693.5000, 695.5000,   0.0000, 681.7000],
        [ 93.1000,  11.8000,  28.4000,   0.0000,   0.0000]])

In [127]:
temp = cost[17]
selected = pi_seq[17][5:10]
temp[selected][:, selected]

tensor([[  0.0000,  43.1000,  59.7000,   0.0000,  31.3000],
        [ 81.3000,   0.0000,  16.6000,   0.0000,  69.6000],
        [ 64.7000,  64.8000,   0.0000,   0.0000,  53.0000],
        [674.7000, 693.5000, 695.5000,   0.0000, 681.7000],
        [ 93.1000,  11.8000,  28.4000,   0.0000,   0.0000]])

In [64]:
a = torch.tensor([])
b = torch.cat((a, torch.LongTensor([1,2,3,4])), dim=0)
torch.cat((b, torch.tensor([1,2,3,4,5,6])), dim=0)
b

tensor([1., 2., 3., 4.])

In [50]:
a = torch.tensor([])
b=torch.cat((a, torch.LongTensor([[1,2,3,4], [4,5,6,7]])), dim=0)#[1:]
b
torch.cat((b, torch.LongTensor([[1,2,3,4], [9,8,7,9]])), dim=0)

tensor([[1., 2., 3., 4.],
        [4., 5., 6., 7.],
        [1., 2., 3., 4.],
        [9., 8., 7., 9.]])

In [9]:
# Original 2D tensor
tensor_2d = torch.tensor([
    [1, 4, 6, 5, 0, 9, 7, 8],
    [3, 0, 2, 6, 1, 7, 5, 4],
    [8, 5, 0, 3, 2, 1, 4, 7]
])

# Function to shift each row from the value 0 onwards
def shift_row(row):
    zero_index = (row == 0).nonzero(as_tuple=True)[0].item()
    part1 = row[zero_index:]  # From value 0 to the end
    part2 = row[:zero_index]  # From the beginning to value 0
    return torch.cat((part1, part2))

# Apply the function to each row and stack the results
new_tensor_2d = torch.stack([shift_row(row) for row in tensor_2d])

print(new_tensor_2d)

tensor([[0, 9, 7, 8, 1, 4, 6, 5],
        [0, 2, 6, 1, 7, 5, 4, 3],
        [0, 3, 2, 1, 4, 7, 8, 5]])


In [57]:
torch.cat([torch.tensor([1,2]), torch.tensor([3,4])], 0)

tensor([1, 2, 3, 4])

In [ ]:
# function used for shifting the tour so that 0 is the starting point (depot)
def shift_row(row, by='min'):
    if by == 'min':
        zero_index = (row.min()).nonzero(as_tuple=True)[0].item()
    elif by == 'max':
        
        zero_index = (row.max()).nonzero(as_tuple=True)[0].item()
    else:
        zero_index = (row == 0).nonzero(as_tuple=True)[0].item()
    part1 = row[zero_index:]  # From value 0 to the end
    part2 = row[:zero_index]  # From the beginning to value 0
    return torch.cat((part1, part2))
        
# if self.problem == 'tsp':
#     ret_sequences = torch.stack([shift_row(row, by='min') for row in ret_sequences])
#     ret_outpus    = torch.stack([shift_row(row, by='max') for row in ret_outpus])

In [70]:
import random
a = [[1,2,3], [4,5,6], [7,8,9]]
b = [4,7,8]
temp = list(zip(location[:2], cost[:2]))
random.shuffle(temp)
res1, res2 = zip(*temp)
# res1 and res2 come out as tuples, and so must be converted to lists.
a, b = list(res1), list(res2)


In [82]:
b[1][:5][:,:5]

tensor([[  0.0000, 630.6000, 649.4000, 655.8000, 653.8000],
        [  0.0000,   0.0000,  18.8000,  25.2000,  23.2000],
        [  0.0000,  49.7000,   0.0000,   6.4000,   4.4000],
        [  0.0000,  53.1000,   6.4000,   0.0000,   2.0000],
        [  0.0000,  54.1000,   4.4000,   2.0000,   0.0000]])

In [128]:
with open('../outputs/tsp_100/tsp100_rollout_20240525T023249/val_pi.pkl', 'rb') as f:
    data = pickle.load(f)

data

tensor([[ 0., 26., 91.,  ...,  3., 13., 19.],
        [ 0., 91., 99.,  ..., 40., 44., 43.],
        [ 0., 54., 55.,  ...,  8.,  9., 10.],
        ...,
        [ 0., 99., 97.,  ..., 39., 40.,  1.],
        [ 0.,  6., 95.,  ..., 44., 45., 12.],
        [ 0.,  1., 80.,  ..., 33., 31., 32.]])

In [39]:
a = torch.tensor([[1,1,2,3], [4,5,6,7],[8,9,8,10]])
b = torch.tensor([])
b = torch.cat((b, a[0]), 0)
b = torch.cat((b, a[1]), 0)
b

tensor([1., 1., 2., 3., 4., 5., 6., 7.])

In [10]:
import pickle
with open('../results/tsp/eval_location/eval_location-tsp_100_tsp100_rollout_20240524T022531-greedy-t1-0-5000.pkl', 'rb') as f:
    data = pickle.load(f)

data


([(1923.6998,
   [0,
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
    16,
    17,
    18,
    19,
    20,
    21,
    22,
    23,
    24,
    25,
    26,
    27,
    28,
    29,
    30,
    31,
    32,
    33,
    34,
    35,
    36,
    37,
    38,
    39,
    40,
    41,
    42,
    43,
    44,
    45,
    46,
    47,
    48,
    49,
    50,
    51,
    52,
    53,
    54,
    55,
    56,
    57,
    58,
    59,
    60,
    61,
    62,
    63,
    64,
    65,
    66,
    67,
    68,
    69,
    70,
    71,
    72,
    73,
    74,
    75,
    76,
    77,
    78,
    79,
    80,
    81,
    82,
    83,
    84,
    85,
    86,
    87,
    88,
    89,
    90,
    91,
    92,
    93,
    94,
    95,
    96,
    97,
    98,
    99],
   8.648553848266602),
  (2328.2998,
   [0,
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
    16,
    17,
    18,
    19,
    20,


In [6]:
def sequence_deviation(output_sequences):
    """
    output_sequences should be a list of list (2 dim)
    return: standard deviation of each sequence
    """
    a = [2*np.sum([np.abs(seq[i+1] - seq[i]) -1 for i in range(len(seq)-1)])/(len(seq) * (len(seq)-1)) for seq in output_sequences]
    return a

sequences = torch.Tensor([[0,1,2,3,4], [4,3,2,1,0], [0,1,3,4,2], [4,2,1,3,0], [1,2,3,4,0]])
#sequences = [[0,1,5]]
sequence_deviation(sequences)

[0.0, 0.0, 0.2, 0.4, 0.3]

In [65]:
p = np.arange(1, 100)
rd.shuffle(p)
a = np.append(0, p)
a

array([ 0,  7, 80, 59, 51, 39, 85, 25, 55, 68, 48, 30,  1, 69, 98, 64, 58,
       18, 63, 47, 19, 76, 37, 90, 89, 22, 42, 61, 96, 49, 62, 14, 32, 60,
       74, 75, 84, 67, 34, 77, 50, 38, 70, 15, 88, 28, 94, 86, 71, 65, 52,
       23, 87, 46, 79, 40, 72,  5, 95,  4, 45,  2, 21, 91, 35, 81, 53, 93,
       41, 56,  9, 36, 44, 97, 26, 16, 24, 82, 12, 11, 27, 20, 33, 43, 17,
        8, 31, 54, 57, 78, 83, 73,  3, 99,  6, 29, 10, 13, 66, 92])

In [105]:
arr = []
for i in range(5):
    p = np.arange(1, 100)
    rd.shuffle(p)
    k = np.append(0, p)
    arr.append(k)
arr

[array([ 0, 85,  2, 51, 41, 64, 84, 19, 16, 48, 74, 88, 27, 55, 86, 49, 18,
        77, 98, 38, 57, 68,  3,  7, 82,  6, 80, 69, 53, 66, 79, 23, 90, 97,
        71, 89, 36,  4, 95, 70, 62, 24, 44, 46, 34,  8, 45, 91, 54, 78, 99,
        13, 22,  5, 92, 39, 31, 52, 83, 42, 25, 65, 32, 81, 28, 76, 21, 72,
        63, 33, 50, 43, 93, 67, 20, 56, 47, 15, 96, 58, 17, 35,  1, 87, 14,
        73, 61, 12, 75, 30, 60,  9, 59, 37, 26, 29, 94, 10, 40, 11]),
 array([ 0, 15, 27, 80,  5, 94, 62, 40, 31, 90, 42, 59, 82, 10, 43, 79, 54,
        35,  4, 28,  7, 56, 63, 23, 61, 51, 75, 95, 46, 77, 25, 67, 53, 33,
        13, 93, 49, 99, 47, 71, 21, 38, 76, 98,  8, 60, 52, 39, 78, 84, 83,
        74,  2, 41, 64, 36, 72, 29,  6, 97,  9, 96, 66, 89, 58, 55, 32, 30,
        17, 19, 57, 11, 45, 91, 87, 14, 12,  1,  3, 70, 16, 88, 50, 20, 73,
        44, 48, 68, 22, 18, 24, 69, 92, 65, 86, 81, 34, 85, 37, 26]),
 array([ 0, 11, 36, 68, 78,  2, 12, 49, 77, 65, 15, 45,  4,  9, 83, 28, 47,
        92, 23, 24, 75, 